<div style="border: 3px solid #4CAF50; padding: 20px; font-size: 24px; font-weight: bold; font-family: 'Arial', sans-serif; text-align: center; line-height: 1.5; background: linear-gradient(to bottom, #e0f7e9, #ffffff); border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
    מטלת סיכום חלק 2: למידת מכונה וכריית נתונים <br>
    :מגישי הפרוייקט <br> 
    יוסף בנקורמונו - 3184555904<br>
    אריאל קורן - 318284239 <br>
    <a href="https://github.com/arielk318/Data-mining-Project/tree/main">קישור לגיטהאב</a> <br>
</div>

In [1]:
import pandas as pd  # ייבוא pandas לעבודה עם מסגרות נתונים
import matplotlib.pyplot as plt  # ייבוא matplotlib ליצירת גרפים
import numpy as np  # ייבוא numpy לחישובים מתמטיים
import urllib.request, urllib.parse, urllib.error  # ייבוא ספריות לעבודה עם בקשות רשת
import requests  # ייבוא requests לשליחת בקשות HTTP
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest
import optuna
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [2]:
df = pd.read_csv("dataset.csv")
# df

In [3]:
na_per_column = df.isnull().sum(axis=0)
na_per_column

manufactor            0
Year                  0
model                 0
Hand                  0
Gear                  1
capacity_Engine      26
Engine_type           5
Prev_ownership      726
Curr_ownership      726
Area                139
City                  0
Price                 0
Pic_num              24
Cre_date              0
Repub_date            0
Description           0
Color               713
Km                   88
Test               1361
Supply_score       1061
dtype: int64

In [4]:
# Function to convert string numbers with commas to float
def convert_to_float(value):
    if value is None or value == 'None':
        return float('nan')
    if isinstance(value, str):
        return float(value.replace(',', ''))
    return value

In [5]:
# Function to replace low values with the mean of the same manufactor and model
def replace_with_mean(group):
    threshold = 600
    # Calculate mean excluding 0 and values below threshold
    mean_value = group[(group['capacity_Engine'] >= threshold)]['capacity_Engine'].mean()
    # Replace values below threshold with the mean, ignoring 0
    group.loc[(group['capacity_Engine'] < threshold) & (group['capacity_Engine'] != 0), 'capacity_Engine'] = mean_value
    return group

In [6]:
## This function imputes missing values using KNN for several columns:
##  ['Engine_type','Year','Hand','Km','Pic_num','capacity_Engine']
def Fill_Engine_type(df):
    # Initialize LabelEncoder for categorical columns
    label_encoder = LabelEncoder()
    
    # Encode categorical columns to numerical values
    df['Engine_type_encoded'] = label_encoder.fit_transform(df['Engine_type'].astype(str))
    
    # Columns to impute
    impute_columns = ['Year','Hand','Km','Pic_num','capacity_Engine', 'Engine_type_encoded']
    
    # Create KNNImputer object
    imputer = KNNImputer(n_neighbors=2)
    
    # Impute missing values
    df[impute_columns] = imputer.fit_transform(df[impute_columns])
    
    # Convert encoded values back to categorical
    df['Engine_type'] = label_encoder.inverse_transform(df['Engine_type_encoded'].astype(int))
    
    # Drop the temporary encoded columns
    df.drop(columns=['Engine_type_encoded'], inplace=True)
    
    return df

In [7]:
# This function impute missing values after encoded columns : manufactor,gear,model.
def Fill_Gear(df):
    # Encode 'manufactor' and 'model' columns
    label_encoder_manufactor = LabelEncoder()
    df['manufactor_encoded'] = label_encoder_manufactor.fit_transform(df['manufactor'].astype(str))

    label_encoder_model = LabelEncoder()
    df['model_encoded'] = label_encoder_model.fit_transform(df['model'].astype(str))

    # Encode 'Gear' column
    label_encoder_gear = LabelEncoder()
    df['Gear_encoded'] = label_encoder_gear.fit_transform(df['Gear'].astype(str))

    # Create KNNImputer object
    imputer = KNNImputer(n_neighbors=2)

    # Impute missing values for encoded columns
    impute_columns = ['manufactor_encoded', 'model_encoded', 'Gear_encoded']
    df[impute_columns] = imputer.fit_transform(df[impute_columns])

    # Convert encoded values back to categorical
    df['manufactor'] = label_encoder_manufactor.inverse_transform(df['manufactor_encoded'].astype(int))
    df['model'] = label_encoder_model.inverse_transform(df['model_encoded'].astype(int))
    df['Gear'] = label_encoder_gear.inverse_transform(df['Gear_encoded'].astype(int))

    # Drop the temporary encoded columns
    df.drop(columns=['manufactor_encoded', 'model_encoded', 'Gear_encoded'], inplace=True)

    return df

In [8]:
def preprocess_data(df):
    # Define categorical columns for OneHotEncoder
    categorical_columns = ['Gear', 'model', 'manufactor', 'Engine_type']
    
    # Drop non-numeric columns that are not needed for OneHotEncoder
    df_numeric = df.drop(categorical_columns, axis=1)
    
    # One-hot encode categorical columns
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')
    encoded_cols = pd.DataFrame(encoder.fit_transform(df[categorical_columns]), columns=encoder.get_feature_names_out(categorical_columns))
    
    # Concatenate numeric and encoded categorical columns
    df = pd.concat([df_numeric, encoded_cols], axis=1)
    
    return df

In [9]:
def prepare_data(df):
    
    # Drop rows with NA in 'Price'
    df = df.dropna(subset=['Price'])
    # remove columns that not neccesery 
    columns_to_drop = ['Prev_ownership','Curr_ownership','Color','Area','City', 'Cre_date', 'Repub_date', 'Description', 'Supply_score','Test']
    df = df.drop(columns=columns_to_drop)
    
    # Convert 'Year', 'Km','capacity_Engine' and 'Pic_num' to numeric
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df['Km'] = df['Km'].apply(convert_to_float)
    df['capacity_Engine'] = df['capacity_Engine'].apply(convert_to_float)
    df['Pic_num'] = df['Pic_num'].apply(convert_to_float)
    
    # If Pic_num is NA fill with 0
    df['Pic_num'].fillna(0, inplace=True)
    # Correcting values of Km column: multiplying values less than 1000 by 1000
    df['Km'] = df['Km'].apply(lambda x: x*1000 if x < 1000 else x)
    
    # Replace special values
    df['Gear'] = df['Gear'].replace({'אוטומט': 'אוטומטית', 'לא מוגדר': np.nan})
    # These scripts will replace all instances of "היבריד" with "היברידי" in the Engine_type column  
    df['Engine_type'] = df['Engine_type'].replace('היבריד', 'היברידי')
    # These scripts will replace all instances of "Lexsus" with "לקסוס" in the manufactor column  
    df['manufactor'] = df['manufactor'].replace('לקסוס','Lexsus')
    # Change the Engine type of the electric car to zero     
    df.loc[df['Engine_type'] == 'חשמלי', 'capacity_Engine'] = 0
    
    # Calculate mean Km by Year
    mean_km_by_year = df.groupby('Year')['Km'].mean()
    # Replace zero Km values with mean values based on Year
    df['Km'] = df.apply(lambda row: mean_km_by_year[row['Year']] if row['Km'] == 0 else row['Km'], axis=1)
    
    # Filter rows where capacity_Engine > 10000 and divide those values by 10 outliers
    df.loc[df['capacity_Engine'] > 10000, 'capacity_Engine'] /= 10
    
    # Apply the function to fillNA at catagorical col(Engine_type) and numerical columns . 
    df = Fill_Engine_type(df)
    
    # Impute missing values after making catagorical columns encoded:
    # ['manufactor','gear','model']
    df = Fill_Gear(df)
    
    ### one hot encoder for catagorical columns: 
    # Apply preprocess_data func on our data- to make One Hot Encoder
    df = preprocess_data(df)
     
    # Define a threshold for low engine capacity values
    threshold = 600

    # Find all columns related to manufactor and model
    manufactor_columns = [col for col in df.columns if col.startswith('manufactor_')]
    model_columns = [col for col in df.columns if col.startswith('model_')]

    # Combine manufactor and model columns to group by
    group_columns = manufactor_columns + model_columns

    # Group by manufactor and model columns and apply the replacement function
    df = df.groupby(group_columns).apply(replace_with_mean).reset_index(drop=True)

    return df


In [10]:
prepare_data(df)

,Year,Hand,capacity_Engine,Price,Pic_num,Km,Gear_אוטומטית,Gear_טיפטרוניק,Gear_ידנית,Gear_רובוטית,...,manufactor_קיה,manufactor_קרייזלר,manufactor_רנו,manufactor_שברולט,Engine_type_בנזין,Engine_type_גז,Engine_type_דיזל,Engine_type_היברידי,Engine_type_חשמלי,Engine_type_טורבו דיזל
0,2015.0,2.0,1600.0,51000.0,2.0,144000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2018.0,1.0,1200.0,49000.0,0.0,69000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2010.0,1.0,1450.0,22500.0,1.0,145000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2016.0,1.0,1600.0,63000.0,5.0,27300.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2012.0,1.0,1248.0,37000.0,1.0,70000.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2016.0,2.0,1400.0,60000.0,1.0,180000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1496,2013.0,4.0,1400.0,45000.0,0.0,160000.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1497,2014.0,1.0,1400.0,30000.0,1.0,131000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1498,2011.0,3.0,1600.0,28000.0,0.0,118000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df = prepare_data(df)
# nan_rows = df[df['Km'].isna()]
# nan_rows
# df['capacity_Engine'].unique()
# missing_values = df.isna().sum().sum()
# print(f"Total missing values: {missing_values}")

In [12]:
na_per_column = df.isnull().sum(axis=0)
na_per_column
 

Year                      0
Hand                      0
capacity_Engine           0
Price                     0
Pic_num                   0
                         ..
Engine_type_גז            0
Engine_type_דיזל          0
Engine_type_היברידי       0
Engine_type_חשמלי         0
Engine_type_טורבו דיזל    0
Length: 329, dtype: int64

In [13]:
# Assuming df_encoded is your DataFrame containing the dataset
# And you want to calculate VIF for all columns except the dependent variable

# Select only numerical columns for VIF calculation
X = df.select_dtypes(include=['number'])

# Add a constant to the model (required by statsmodels for calculating VIF)
X = sm.add_constant(X)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Print the VIF values
print(vif_data)

C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


                    Feature        VIF
0                     const   0.000000
1                      Year   9.263204
2                      Hand   2.434198
3           capacity_Engine   9.338758
4                     Price   6.659879
..                      ...        ...
325          Engine_type_גז  18.660589
326        Engine_type_דיזל  25.587624
327     Engine_type_היברידי  35.342000
328       Engine_type_חשמלי   5.060306
329  Engine_type_טורבו דיזל   5.886251

[330 rows x 2 columns]


In [14]:
# Drop features with high VIF
threshold = 5
# threshold = 10
features_to_drop = vif_data[vif_data['VIF'] > threshold]['Feature']
df_reduced = df.drop(columns=features_to_drop)
print(df_reduced)

      Hand  Pic_num        Km  model_ A3  model_ A4  model_ A5  model_ A6  \
0      2.0      2.0  144000.0        0.0        0.0        0.0        0.0   
1      1.0      0.0   69000.0        0.0        0.0        0.0        0.0   
2      1.0      1.0  145000.0        0.0        0.0        0.0        0.0   
3      1.0      5.0   27300.0        0.0        0.0        0.0        0.0   
4      1.0      1.0   70000.0        0.0        0.0        0.0        0.0   
...    ...      ...       ...        ...        ...        ...        ...   
1495   2.0      1.0  180000.0        0.0        0.0        0.0        0.0   
1496   4.0      0.0  160000.0        0.0        0.0        0.0        0.0   
1497   1.0      1.0  131000.0        0.0        0.0        0.0        0.0   
1498   3.0      0.0  118000.0        0.0        0.0        0.0        0.0   
1499   3.0      1.0  163899.0        0.0        0.0        0.0        0.0   

      model_ Q3  model_ R8  model_ RS5  ...  model_אאודי A3  model_אאודי A4

<h1>Machine learning model</h1>

In [15]:
# Select relevant features
#categorical_features = ['manufactor','model','Gear', 'Engine_type']
#relevant_features = ['Year','Hand','Pic_num','Km', 'capacity_Engine'] + [col for col in df.columns if col.startswith(tuple(categorical_features))]
#X = df[relevant_features]
#y = df['Price']
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()
relevant_features = numerical_features + categorical_features

poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(df[numerical_features])
df_poly = pd.DataFrame(X_poly, columns=poly.get_feature_names_out (numerical_features))

# Choose 50% of the polonomial feature in a random way .
np.random.seed(42)
selected_features = np.random.choice(df_poly.columns, size=int(len(df_poly.columns) * 0.5), replace=False)
df_poly_reduced = df_poly[selected_features]

X = df_poly_reduced
y = df['Price']

# Remove any remaining rows with NaN values
X = X.dropna()
y = y[X.index]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<h1> Elastic Net model


In [18]:
#This code uses Optuna to perform hyperparameter optimization for an ElasticNet model by conducting 100 trials with different hyperparameter values. 
#It trains the model with the best hyperparameters, evaluates its performance using various performance metrics, 
#and performs 10-fold cross-validation to assess the stability of the performance.

In [19]:
# we download : pip install optuna to optimise the model
# Define the model
elastic_net_model = ElasticNet()

# Objective function for optimization
def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 0.001, 0.1)
    l1_ratio = trial.suggest_uniform('l1_ratio', 0.0, 1.0)
    
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Create study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

# Best parameters from Optuna
best_params = study.best_params
best_alpha = best_params['alpha']
best_l1_ratio = best_params['l1_ratio']

# Fit the model with the best parameters
best_elastic_net_model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio)
best_elastic_net_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = best_elastic_net_model.predict(X_test_scaled)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Best Parameters: {best_params}")
print(f"Mean Squared Error: {mse}")
print(f'RMSE: {rmse}')
print(f"R-squared Score: {r2}")


[I 2024-07-20 19:03:20,645] A new study created in memory with name: no-name-c3d232d5-6431-4a15-9ec0-70c40d984642
C:\Users\User\AppData\Local\Temp\ipykernel_1500\2013623720.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.001, 0.1)
C:\Users\User\AppData\Local\Temp\ipykernel_1500\2013623720.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  l1_ratio = trial.suggest_uniform('l1_ratio', 0.0, 1.0)
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regu

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.184e+10, tolerance: 5.745e+07
  model = cd_fast.enet_coordinate_descent(
[I 2024-07-20 19:04:24,482] Trial 7 finished with value: 68164372.39611194 and parameters: {'alpha': 0.03774988889057, 'l1_ratio': 0.43195068715445994}. Best is trial 2 with value: 68064283.06387779.
C:\Users\User\AppData\Local\Temp\ipykernel_1500\2013623720.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.001, 0.1)
C:\Users\User\AppData\Local\Temp\ipykernel_1500\2013623720.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.130e+09, tolerance: 5.745e+07
  model = cd_fast.enet_coordinate_descent(
[I 2024-07-20 19:05:34,993] Trial 14 finished with value: 68795269.07051845 and parameters: {'alpha': 0.016188289754122058, 'l1_ratio': 0.3858082135020971}. Best is trial 2 with value: 68064283.06387779.


Best Parameters: {'alpha': 0.03082462965960418, 'l1_ratio': 0.3762726973358853}
Mean Squared Error: 68064283.06387779
RMSE: 8250.108063769698
R-squared Score: 0.8606938807067579


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.143e+10, tolerance: 5.745e+07
  model = cd_fast.enet_coordinate_descent(


In [20]:
markdown_output = f"""
 Model performance :
- MSE: `{mse}`
- RMSE: `{rmse}`
- R-squared Score: `{r2}`
"""
print(markdown_output)



 Model performance :
- MSE: `68064283.06387779`
- RMSE: `8250.108063769698`
- R-squared Score: `0.8606938807067579`




<h4 style="color:black;font-weight:bold;"> Perform 10-fold cross-validation and report the performance metrics</h4>


In [21]:
#10-fold cross-validation is a technique where the dataset is divided into 10 equal parts (folds). 
#The model is trained on 9 folds and tested on the remaining fold, and this process is repeated 10 times, with each fold serving as the test set once. 
#The results are averaged to provide a robust estimate of the model's performance.

#Positive and Negative R-squared Values:
#Positive R-squared values indicate that the model explains the variability in the data well.
#Negative R-squared values indicate that the model performs worse than a simple average guess (i.e., the mean of the target values).

In [22]:
# Perform 10-fold cross-validation and report the performance metrics
cv_scores = cross_val_score(best_elastic_net_model, scaler.transform(X), y, cv=10, scoring='r2')
print(f'10-fold cross-validation R-squared scores: {cv_scores}')
print(f'Mean R-squared score: {cv_scores.mean()}')

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.460e+10, tolerance: 6.474e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.425e+10, tolerance: 6.566e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.458e+10, tolerance: 

10-fold cross-validation R-squared scores: [0.85875928 0.86502137 0.87190352 0.87826866 0.8297342  0.88336706
 0.77625525 0.86479575 0.80240682 0.87862811]
Mean R-squared score: 0.8509140020260112


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.444e+10, tolerance: 6.455e+07
  model = cd_fast.enet_coordinate_descent(


In [23]:
# Identify the five most impactful features
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': best_elastic_net_model.coef_})
coefficients['Absolute_Coefficient'] = coefficients['Coefficient'].abs()
top_features = coefficients.sort_values(by='Absolute_Coefficient', ascending=False).head(5)
print("Top 5 features with the highest impact:")
print(top_features[['Feature', 'Coefficient']])

Top 5 features with the highest impact:
                    Feature  Coefficient
20222              Price Km  9991.398246
18107                  Year  7822.119985
26860  Price manufactor_קיה  6111.878896
5708   Km Engine_type_בנזין -5081.422425
18939      Km Gear_אוטומטית -4916.963836
